## Download Hosted Feature Services as FGDB

#### Import modules and connect to the GIS

In [ ]:
from arcgis.gis import GIS
import datetime as dt
username = input("Input your username: ")
gis = GIS("https://arcgis.com", username)

#### Create the search query to find a specific number of the signed-in user's feature services and define the folder location to store the backups

- Note, there are many more options for searching for content using the ArcGIS API for Python. Usage notes and a full list of parameters can be found [here](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.search). 
- No need to escape `\` in the file path format, as this is converted to `\\` for use in the function. For example, you may enter `C:\users\username\documents\arcgis_backups`. 

In [ ]:
folder_path = input("Please enter the file location to store the backups: ")
num_items = int(input("How many items do you want to back up? "))
query_string = "type:Feature Service, owner:{}".format(username)
items = gis.content.search(query=query_string, max_items=num_items, sort_field='modifed', sort_order='desc')
print(str(len(items)) + " items will be backed up to " + folder_path +". See the list below:")
items

#### Create a function to back up hosted feature layers

- The function excludes hosted feature layer views to avoid duplicate backups.

In [ ]:
def download_as_fgdb(item_list, backup_location):
    for item in item_list:
        try:
            if 'View Service' in item.typeKeywords:
                print(item.title + " is view, not downloading")
            else: 
                print("Downloading " + item.title)
                version = dt.datetime.now().strftime("%d_%b_%Y")
                result = item.export(item.title + "_" + version, "File Geodatabase")
                result.download(backup_location)
                result.delete()
                print("Successfully downloaded " + item.title)
        except:
            print("An error occurred downloading " + item.title)
    print("The function has completed")

#### Call the function on the item list with your backup location

In [ ]:
download_as_fgdb(items, folder_path)